In [2]:
import cv2
import numpy as np
import os
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

# Define the source and target directory
source_directory = './database'
target_directory = './database_binarized'

# Create target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

f1_scores = []

# Process each image
for i in range(1, 21):  # Assuming images are named from 1.pgm to 20.pgm
    original_image_path = f'{source_directory}/{i}.pgm'
    ground_truth_image_path = f'{source_directory}/{i}_gt.pgm'
    target_image_path = f'{target_directory}/{i}.pgm'
    
    # Check if the ground truth image exists for the current image
    if not os.path.exists(ground_truth_image_path):
        print(f"Ground truth for image {i} does not exist. Skipping...")
        continue

    # Load the original and ground truth images
    original_image = cv2.imread(original_image_path, 0)
    ground_truth_image = cv2.imread(ground_truth_image_path, 0)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(original_image, (9, 9), 0)

    # Adaptive Gaussian Thresholding after blurring
    binary_adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)

    # Use morphological closing to close small holes, then opening to remove noise
    kernel = np.ones((1,1), np.uint8)
    closing = cv2.morphologyEx(binary_adaptive, cv2.MORPH_CLOSE, kernel, iterations=3)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=3)

    # Apply the median filter after morphological operations
    median_filtered = cv2.medianBlur(opening, 7)

    # Invert colors at the end as requested
    inverted_binary = cv2.bitwise_not(median_filtered)

    # Flatten the images for F1 score calculation
    inverted_binary_flatten = inverted_binary.flatten() / 255
    ground_truth_flatten = ground_truth_image.flatten() / 255

    # Calculate F1 score
    f1score_inverted = f1_score(ground_truth_flatten, inverted_binary_flatten)
    f1_scores.append(f1score_inverted)

    # Save the processed image to the target directory
    cv2.imwrite(target_image_path, inverted_binary)



In [3]:
import numpy as np
import cv2
import os

def calculate_metrics(gt, pred):
    TP = np.sum((gt == 255) & (pred == 255))
    TN = np.sum((gt == 0) & (pred == 0))
    FP = np.sum((gt == 0) & (pred == 255))
    FN = np.sum((gt == 255) & (pred == 0))

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    f1_score = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) != 0 else 0

    return accuracy, sensitivity, specificity, precision, f1_score

folder_path = './database'
folder_path2 = './database_binarized'

metrics = []

for i in range(1, 21):
    gt_path = os.path.join(folder_path, f'{i}_gt.pgm')
    bn_path = os.path.join(folder_path2, f'{i}.pgm')
    
    if os.path.exists(gt_path) and os.path.exists(bn_path):
        gt = cv2.imread(gt_path, 0)
        pred = cv2.imread(bn_path, 0)
        
        metrics.append(calculate_metrics(gt, pred))
    else:
        metrics.append((None, None, None, None, None))

print("{:<6} | {:<9} | {:<11} | {:<11} | {:<9} | {:<7}".format("Image", "Accuracy", "Sensitivity", "Specificity", "Precision", "F1 Score"))
print("-" * 70)  # Imprime una línea separadora

for i, (accuracy, sensitivity, specificity, precision, f1_score) in enumerate(metrics, 1):
    print("{:<6} | {:<9.3f} | {:<11.3f} | {:<11.3f} | {:<9.3f} | {:<7.3f}".format(i, accuracy or 0, sensitivity or 0, specificity or 0, precision or 0, f1_score or 0))


Image  | Accuracy  | Sensitivity | Specificity | Precision | F1 Score
----------------------------------------------------------------------
1      | 0.962     | 0.783       | 0.970       | 0.533     | 0.634  
2      | 0.953     | 0.870       | 0.957       | 0.501     | 0.636  
3      | 0.961     | 0.844       | 0.966       | 0.524     | 0.646  
4      | 0.961     | 0.486       | 0.983       | 0.564     | 0.522  
5      | 0.947     | 0.735       | 0.955       | 0.382     | 0.503  
6      | 0.944     | 0.803       | 0.949       | 0.374     | 0.511  
7      | 0.959     | 0.725       | 0.971       | 0.567     | 0.636  
8      | 0.947     | 0.441       | 0.971       | 0.430     | 0.436  
9      | 0.949     | 0.822       | 0.956       | 0.499     | 0.621  
10     | 0.946     | 0.664       | 0.959       | 0.429     | 0.521  
11     | 0.932     | 0.688       | 0.943       | 0.337     | 0.452  
12     | 0.947     | 0.837       | 0.956       | 0.621     | 0.713  
13     | 0.961     | 0.851     